## 4A. Práctica: Detección de Plagio
### Nombre: **Luis Fernando Izquierdo Berdugo**
### Materia: **Procesamiento de Información**
### Fecha: **28 de Octubre de 2024**



Instrucciones:

En esta actividad, el ejercicio propuesto es: **detectar a nivel básico, por medio de una medida de similitud, el plagio que hay dentro de un conjunto de documentos**.

Los datos son documentos de noticias, adaptados a la tarea de detección de plagio. En el conjunto de datos del directorio  `suspicious-documents`, hay  documentos que tienen fragmentos plagiados de los documentos fuente `source-documents`, sin embargo, también hay otros documentos que simularon plagio, pero no se plagió ningún párrafo de los documentos fuente.

La tarea es encontrar  el nivel de plagio entre los archivos del directorio `source-documents` y los archivos del directorio `suspicious-documents`.

Entregar un notebook con el código fuente para obtener la similitud entre dos documentos. Tomar 20 documentos de source-documents y por cada uno, encontrar los 5 archivos de suspicious-documents más parecidos. Indicar los nombres y la similitud obtenida.

Aplicar los preprocesamientos: usar solo letras minúsculas, remover stopwords (palabras que no son de contenido)  y aplicar un proceso de stemming para reducir el vocabulario.
 

Nota: Usar la lista de stopwords para eliminarlas, recordar si se le aplicó stemming, aplicar stemming también a las stopwords, debido a que alguna palabra podría cambiar.

1. Se recomienda usar el stemmer de NLTK. Para facilidad, usar Porter (inglés).

http://www.nltk.org/howto/stem.html

Básicamente usar:

from nltk.stem.porter import *

stemmer = PorterStemmer()

word = stemmer.stem('pages')

Word contendrá la palabra normalizada por medio de stemming ('page')  aplicar ese proceso a todos los textos. 

2. Para procesar la medida de similitud, usar la medida Jaccard y Dice

In [8]:
import os
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# Descargar stopwords si no están descargadas
nltk.download('stopwords')

# Crear una instancia del PorterStemmer
stemmer = PorterStemmer()

# Definir la lista de stopwords y aplicar stemming
stop_words = set(stopwords.words('english'))
stemmed_stop_words = {stemmer.stem(word) for word in stop_words}

def preprocess_text(text):
    """Preprocesa el texto: minúsculas, eliminar stopwords y aplicar stemming."""
    words = text.lower().split()
    stemmed_words = [stemmer.stem(word) for word in words if word not in stemmed_stop_words]
    return ' '.join(stemmed_words)

def jaccard_similarity(doc1, doc2):
    """Calcula la similitud de Jaccard entre dos documentos."""
    words_doc1 = set(doc1.split())
    words_doc2 = set(doc2.split())
    intersection = words_doc1.intersection(words_doc2)
    union = words_doc1.union(words_doc2)
    return len(intersection) / len(union)

def dice_similarity(doc1, doc2):
    """Calcula la similitud de Dice entre dos documentos."""
    words_doc1 = set(doc1.split())
    words_doc2 = set(doc2.split())
    intersection = words_doc1.intersection(words_doc2)
    return 2 * len(intersection) / (len(words_doc1) + len(words_doc2))

def read_documents(directory, limit=None):
    """Lee todos los documentos de un directorio y devuelve un diccionario con el nombre del archivo y su contenido."""
    documents = {}
    filenames = sorted([f for f in os.listdir(directory) if f.endswith(".txt")])
    for i, filename in enumerate(filenames):
        if limit is not None and i >= limit:
            break
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            documents[filename] = file.read()
    return documents

# Rutas de los directorios
source_dir = "/Users/izluis/Documents/source-documents"
suspicious_dir = "/Users/izluis/Documents/suspicious-documents"

# Leer y preprocesar los documentos
source_documents = read_documents(source_dir, limit=20)
suspicious_documents = read_documents(suspicious_dir)

preprocessed_source_docs = {name: preprocess_text(text) for name, text in source_documents.items()}
preprocessed_suspicious_docs = {name: preprocess_text(text) for name, text in suspicious_documents.items()}

# Calcular similitudes y encontrar los documentos más similares
results = {}

for source_name, source_text in preprocessed_source_docs.items():
    similarities = []
    for suspicious_name, suspicious_text in preprocessed_suspicious_docs.items():
        jaccard_sim = jaccard_similarity(source_text, suspicious_text)
        dice_sim = dice_similarity(source_text, suspicious_text)
        similarities.append((suspicious_name, jaccard_sim, dice_sim))
    
    # Ordenar por similitud de Jaccard y tomar los 5 más similares
    similarities.sort(key=lambda x: x[1], reverse=True)
    results[source_name] = similarities[:5]

# Mostrar resultados
for source_name, similar_docs in results.items():
    print(f"Documentos más similares a {source_name}:")
    for doc_name, jaccard_sim, dice_sim in similar_docs:
        print(f"  {doc_name}: Jaccard={jaccard_sim:.4f}, Dice={dice_sim:.4f}")
    print()

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


Documentos más similares a source-document0001.txt:
  suspicious-document2121.txt: Jaccard=0.1255, Dice=0.2231
  suspicious-document2205.txt: Jaccard=0.1246, Dice=0.2216
  suspicious-document2048.txt: Jaccard=0.1239, Dice=0.2205
  suspicious-document1267.txt: Jaccard=0.1238, Dice=0.2203
  suspicious-document2167.txt: Jaccard=0.1231, Dice=0.2192

Documentos más similares a source-document0002.txt:
  suspicious-document0020.txt: Jaccard=0.1050, Dice=0.1900
  suspicious-document0019.txt: Jaccard=0.1034, Dice=0.1874
  suspicious-document1058.txt: Jaccard=0.0863, Dice=0.1588
  suspicious-document1088.txt: Jaccard=0.0861, Dice=0.1586
  suspicious-document1120.txt: Jaccard=0.0850, Dice=0.1567

Documentos más similares a source-document0003.txt:
  suspicious-document0030.txt: Jaccard=0.1535, Dice=0.2661
  suspicious-document0029.txt: Jaccard=0.1327, Dice=0.2343
  suspicious-document0027.txt: Jaccard=0.0867, Dice=0.1596
  suspicious-document0021.txt: Jaccard=0.0866, Dice=0.1593
  suspicious-doc